# Train a sensor processing model using a Convolutional Variational Autoencoder 

Using the Julian-8897-Conv-VAE-PyTorch implementation to train a sensor processing model based on convolutional variational autoencoder. 

The parameters of the training are described by an experiment run of type "sensorprocessing_conv_vae". The result of runing the code in this notebook is the model files that are stored in the experiment directory. 

As the model files will have unpredictable date-time dependent names, after running a satisfactory model, the mode name and directory will need to be copied to the experiment/run yaml file, in the model_subdir and model_checkpoint fields.


In [1]:
import sys
sys.path.append("..")
from settings import Config
import pathlib
from pprint import pprint
import shutil

# adding the Julian-8897-Conv-VAE-PyTorch into the path
sys.path.append(Config()["conv_vae"]["code_dir"])

# At some point in the development, this hack was necessary for some reason. 
# It seems that as of Feb 2025, the code runs on Windows and Linux without it.
#temp = pathlib.PosixPath
#pathlib.PosixPath = pathlib.WindowsPath

from conv_vae import get_conv_vae_config, create_configured_vae_json, train

Loading pointer config file: C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
Loading machine-specific config file: G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml


In [2]:
# If it is set to true, no actual copying will be done
dry_run = False

# Specify and load the experiment
experiment = "sensorprocessing_conv_vae"
run = "proprio_128" 
exp = Config().get_experiment(experiment, run)
pprint(exp)

Configuration for experiment: sensorprocessing_conv_vae/proprio_128 successfully loaded
{'data_dir': WindowsPath('c:/Users/lboloni/Documents/Code/_TempData/BerryPicker-experiments/sensorprocessing_conv_vae/proprio_128'),
 'epochs': 300,
 'exp_run_sys_dep_file': WindowsPath('G:/My Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/experiment-config/LotziYoga/sensorprocessing_conv_vae/proprio_128_sysdep.yaml'),
 'exp_run_sys_indep_file': WindowsPath('C:/Users/lboloni/Documents/Code/_Checkouts/BerryPicker/src/experiment_configs/sensorprocessing_conv_vae/proprio_128.yaml'),
 'group_name': 'sensorprocessing_conv_vae',
 'json_template_name': 'conv-vae-config-default.json',
 'latent_size': 128,
 'model_checkpoint': 'checkpoint-epoch200.pth',
 'model_dir': 'models',
 'model_name': 'VAE_Robot',
 'model_subdir': '0222_160734',
 'run_name': 'proprio_128',
 'save_period': 5,
 'training_data_dir': 'vae-training-data',
 'training_task': 'proprio_sp_training',
 'validation_demo': '2024_12_26_

### Create the training data for the Conv-VAE

We collect the training data for the Conv-VAE by gathering all the pictures from all the demonstrations of a specific task. One can select the pictures by creating a specific task, and copy there all the relevant demonstrations. 

The collected pictures are put in a newly created training directory for the run:

```
$experiment\vae-training-data\Images\*.jpg
```

In [3]:
def copy_images_to_training_dir(taskname, training_image_dir):
    """Copy all the images from a specific task into the training image dir."""
    task_dir = pathlib.Path(demos_dir, taskname)
    # _, task_dir = ui_choose_task(offer_task_creation=True)

    for demo in task_dir.iterdir():
        if not demo.is_dir(): continue
        for item in demo.iterdir():
            if item.suffix != ".jpg": continue
            name = f"{demo.name}_{item.stem}.jpg"
            destination = pathlib.Path(training_image_dir, name)
            print(f"copy {item} to \n{destination}")
            if not dry_run:
                shutil.copyfile(item, destination)

In [4]:
demos_top = pathlib.Path(Config()["demos"]["directory"])
demos_dir = pathlib.Path(demos_top, "demos")

subdir_count = sum(1 for item in demos_dir.iterdir() if item.is_dir())
print(f"Number of demo directories: {subdir_count}")

# Deciding on the location of the training data
training_data_dir = pathlib.Path(exp["data_dir"], exp["training_data_dir"])
# training_data_dir = pathlib.Path(Config()["conv_vae"]["training_data_dir"])
training_image_dir = pathlib.Path(training_data_dir, "Images")
training_image_dir.mkdir(exist_ok = False, parents=True)

print(f"Training data dir={training_image_dir}")

# Define a set of common image file extensions
image_extensions = {".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".webp"}
# Count the image files
image_count = sum(1 for item in training_image_dir.iterdir() if item.suffix.lower() in image_extensions and item.is_file())

print(f"Number of image files in training dir: {image_count}")

if image_count == 0:
    taskname = exp['training_task']
    copy_images_to_training_dir(
        taskname = taskname, training_image_dir=training_image_dir)
else:
    print("There are already images in training image dir {training_image_dir}. Do not repeat the copying.")            


Number of demo directories: 17
Training data dir=c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\sensorprocessing_conv_vae\proprio_128\vae-training-data\Images
Number of image files in training dir: 0
copy C:\Users\lboloni\Documents\Code\_TempData\BerryPicker-demos\demos\proprio_sp_training\2025_02_25__15_39_35\00001_dev2.jpg to 
c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\sensorprocessing_conv_vae\proprio_128\vae-training-data\Images\2025_02_25__15_39_35_00001_dev2.jpg
copy C:\Users\lboloni\Documents\Code\_TempData\BerryPicker-demos\demos\proprio_sp_training\2025_02_25__15_39_35\00002_dev2.jpg to 
c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\sensorprocessing_conv_vae\proprio_128\vae-training-data\Images\2025_02_25__15_39_35_00002_dev2.jpg
copy C:\Users\lboloni\Documents\Code\_TempData\BerryPicker-demos\demos\proprio_sp_training\2025_02_25__15_39_35\00003_dev2.jpg to 
c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-expe

# Run the training

Actually run the training. This is done by creating the json-based configuration file of the Conv-VAE library with the parameters specified in the library. Then we call the code of the library to perform the training. 

In [5]:
# Create the vae configuration, based on the experiment
file = create_configured_vae_json(exp)
print(file)
vae_config = get_conv_vae_config(file)

# actually run the training
print(f'Running the trainer from scratch for {vae_config["trainer"]["epochs"]}')
trainer = train(vae_config)

C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\sensorprocessing\conv-vae-config-default.json
{'name': 'VAE_Robot', 'n_gpu': 1, 'arch': {'type': 'VanillaVAE', 'args': {'in_channels': 3, 'latent_dims': 128, 'flow': False}}, 'data_loader': {'type': 'CelebDataLoader', 'args': {'data_dir': 'c:\\Users\\lboloni\\Documents\\Code\\_TempData\\BerryPicker-experiments\\sensorprocessing_conv_vae\\proprio_128\\vae-training-data', 'batch_size': 64, 'shuffle': True, 'validation_split': 0.2, 'num_workers': 2}}, 'optimizer': {'type': 'Adam', 'args': {'lr': 0.005, 'weight_decay': 0.0, 'amsgrad': True}}, 'loss': 'elbo_loss', 'metrics': [], 'lr_scheduler': {'type': 'StepLR', 'args': {'step_size': 50, 'gamma': 0.1}}, 'trainer': {'epochs': 300, 'save_dir': 'c:\\Users\\lboloni\\Documents\\Code\\_TempData\\BerryPicker-experiments\\sensorprocessing_conv_vae\\proprio_128\\models', 'save_period': 5, 'verbosity': 2, 'monitor': 'min val_loss', 'early_stop': 10, 'tensorboard': True}}
c:\Users\lboloni\Doc

C:\Users\lboloni\Documents\Code\_Checkouts\Conv-VAE-PyTorch\utils\util.py:59: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self._data.total[key] += value * n
C:\Users\lboloni\Documents\Code\_Checkouts\Conv-VAE-PyTorch\utils\util.py:60: Futu

In [6]:
# These are the metrics recorded
# they are of utils/util.py / MetricTracker which has a pandas dataframe as data
print(trainer.train_metrics)
print(trainer.valid_metrics)
# 
trainer.train_metrics._data
# trainer.valid_metrics._data

,total,counts,average
loss,58320.796631,13,4486.215125


__Important__ After the training finished, in order to use the resulting system, one need to edit the run file (eg: vae_01.yaml) and enter into it the location of the checkpoint. This is the content printed by the code cell below

In [7]:
print(f'The following next to be put either into \n the system dependent file {exp["exp_run_sys_dep_file"]} \n or the system independent file {exp["exp_run_sys_indep_file"]}')

print(f"model_subdir: '{trainer.checkpoint_dir.name}'")
print(f"model_checkpoint: 'checkpoint-epoch{trainer.epochs}.pth'")


The following next to be put either into 
 the system dependent file G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\sensorprocessing_conv_vae\proprio_128_sysdep.yaml 
 or the system independent file C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\sensorprocessing_conv_vae\proprio_128.yaml
model_subdir: '0227_211544'
model_checkpoint: 'checkpoint-epoch300.pth'


In [8]:
print(exp)

{'training_data_dir': 'vae-training-data', 'json_template_name': 'conv-vae-config-default.json', 'model_name': 'VAE_Robot', 'model_dir': 'models', 'latent_size': 128, 'epochs': 300, 'save_period': 5, 'group_name': 'sensorprocessing_conv_vae', 'training_task': 'proprio_sp_training', 'validation_task': 'proprio_sp_validation', 'validation_demo': '2024_12_26__16_44_06', 'model_subdir': '0222_160734', 'model_checkpoint': 'checkpoint-epoch200.pth', 'run_name': 'proprio_128', 'data_dir': WindowsPath('c:/Users/lboloni/Documents/Code/_TempData/BerryPicker-experiments/sensorprocessing_conv_vae/proprio_128'), 'exp_run_sys_indep_file': WindowsPath('C:/Users/lboloni/Documents/Code/_Checkouts/BerryPicker/src/experiment_configs/sensorprocessing_conv_vae/proprio_128.yaml'), 'exp_run_sys_dep_file': WindowsPath('G:/My Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/experiment-config/LotziYoga/sensorprocessing_conv_vae/proprio_128_sysdep.yaml')}
